# **Importamos las dependencias**

In [1]:
# --- Importaciones para Manejo de Archivos y Directorios ---
import os
import sys
import subprocess
import re

from google.cloud import storage
from typing import List, Dict, Any

import math

# --- Importamos Vertex AI ---
import vertexai


# --- Importaciones para Visualización y Formato de Texto ---

# Se utiliza para mostrar contenido enriquecido, como texto con formato Markdown,
# directamente en entornos como Jupyter Notebooks o IPython.
from IPython.display import Markdown, display

# Importa la clase `Markdown` de la biblioteca `rich`, que sirve para renderizar
# Markdown con formato avanzado en la terminal. Se le da un alias `RichMarkdown`
# para evitar conflictos de nombre con la importación anterior.
from rich.markdown import Markdown as RichMarkdown


# --- Importaciones para el Modelo Generativo de Vertex AI ---

# Importa las clases necesarias del SDK de Vertex AI para interactuar con los modelos generativos.
# - GenerationConfig: Para configurar los parámetros de la respuesta (ej. temperatura, top_p).
# - GenerativeModel: La clase principal para cargar y usar un modelo generativo como Gemini.
# - Image: Para manejar y enviar imágenes como parte de la entrada al modelo (enfoque multimodal).
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image

In [2]:
# --- Para el proceso de datos y visualización ---
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import time


# --- Para manejar las variables de entorno ---
import os
from dotenv import load_dotenv


# --- Desactiva las advertencias de asignaciones encadenadas en pandas para evitar mensajes de warning al modificar DataFrames.
pd.options.mode.chained_assignment = None  # default='warn'


# --- Dependencias de Vertex AI ---
import vertexai                                              # Importa el módulo principal de Vertex AI.
from vertexai import init                                    # Inicializa Vertex AI con las credenciales y configuraciones necesarias.
from vertexai.vision_models import Image as VMImage          # Importa la clase Image de Vertex AI para manejar imágenes.
from vertexai.vision_models import MultiModalEmbeddingModel  # Importa el modelo de embeddings multimodales de Vertex AI para procesar imágenes y videos.
from vertexai.vision_models import Video                     # Clase para manejar archivos de video en Vertex AI.
from vertexai.vision_models import VideoSegmentConfig        # Configuración para segmentar videos
from vertexai.generative_models import GenerativeModel       # Importa la clase para modelos generativos, como Gemini.
from vertexai.generative_models import Part                  # Importa la clase Part para manejar partes de un mensaje, como texto o imágenes.


# --- Para conectarse y consultar un endpoint de búsqueda vectorial (Vector Search) en Vertex AI. 
from google.cloud.aiplatform.matching_engine import MatchingEngineIndexEndpoint 

# --- Para acceder a los buckets de Google Cloud Storage y manejar archivos.
from google.cloud import storage


# --- Dependencias para poder visualizar ---
from IPython.display import Video as MVideo                  # Permite mostrar videos directamente en celdas de Jupyter Notebook.
from IPython.display import HTML                             # Permite mostrar contenido HTML en celdas de Jupyter Notebook.
from IPython.display import Image as ImageByte               # Permite mostrar imágenes en el notebook (renombrado como ImageByte para evitar conflictos de nombres).
from IPython.display import display                          # Función general para mostrar objetos en el notebook.
from sklearn.metrics.pairwise import cosine_similarity       # Función para calcular la similitud coseno entre vectores, útil para comparar embeddings.

In [3]:
from vertexai.generative_models import GenerationConfig
from vertexai.generative_models import GenerationResponse

# **Configuración de las credenciales**

In [4]:
# --- Carga las Variables de Entorno ---
load_dotenv()

PROJECT_ID = os.getenv("PROJECT_ID")                        # ID del proyecto de Google Cloud
LOCATION = os.getenv("LOCATION")                            # Región donde se encuentran los recursos de Vertex AI
INDEX_ID = os.getenv("INDEX_ID")                            # ID del índice de búsqueda vectorial en Vertex AI
ENDPOINT_ID = os.getenv("ENDPOINT_ID")                      # ID del endpoint de búsqueda vectorial en Vertex AI
BUCKET_NAME = os.getenv("BUCKET_NAME")                      # Nombre del bucket de Google Cloud Storage donde se almacenan los videos
VIDEO_FOLDER_PATH = os.getenv("VIDEO_FOLDER_PATH")          # Ruta del folder dentro del bucket donde se encuentran los videos

# Verificamos que las variebles de entorno esten bien
# print(f"Proyecto: {PROJECT_ID}, Ubicación: {LOCATION}, Índice: {INDEX_ID}, Endpoint: {ENDPOINT_ID}, Bucket: {BUCKET_NAME}, Ruta de Videos: {VIDEO_FOLDER_PATH}")

python-dotenv could not parse statement starting at line 3
python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 5
python-dotenv could not parse statement starting at line 7
python-dotenv could not parse statement starting at line 15
python-dotenv could not parse statement starting at line 16
python-dotenv could not parse statement starting at line 17
python-dotenv could not parse statement starting at line 19
python-dotenv could not parse statement starting at line 28
python-dotenv could not parse statement starting at line 29
python-dotenv could not parse statement starting at line 30
python-dotenv could not parse statement starting at line 32
python-dotenv could not parse statement starting at line 33


In [5]:
# Iniciamos Vertex AI con el proyecto y la ubicación especificados.
init(project = PROJECT_ID, location = LOCATION)

In [6]:
# --- Inicialización del LLM ---
# model = GenerativeModel('gemini-2.5-flash') # Debo probar con varios modelos, para ver cual tiene mejor rendimiento.

# **Funciones**

In [7]:
def obtener_nombres_de_archivos(nombre_bucket: str, nombre_carpeta: str) -> List[str]:
    """
    Obtiene una lista con los nombres de los archivos de una "carpeta" en un bucket.

    Args:
        nombre_bucket: El nombre del bucket de Cloud Storage.
        nombre_carpeta: La ruta de la carpeta (prefijo) dentro del bucket.

    Returns:
        Una lista de strings con los nombres de los archivos.
        Retorna una lista vacía si la carpeta no existe o hay un error.
    """
    try:
        cliente_storage = storage.Client()
        nombres_archivos = []

        # Aseguramos que el prefijo termine con '/' para que actúe como una carpeta
        if nombre_carpeta and not nombre_carpeta.endswith('/'):
            nombre_carpeta += '/'

        # Usamos el parámetro 'prefix' para filtrar por la carpeta
        blobs = cliente_storage.list_blobs(nombre_bucket, prefix=nombre_carpeta)

        for blob in blobs:
            # Obtiene el nombre del archivo relativo a la carpeta que estamos viendo
            nombre_relativo = blob.name.replace(nombre_carpeta, '', 1)
            if nombre_relativo: # Evita añadir una entrada vacía si la carpeta misma aparece
                nombres_archivos.append(nombre_relativo)
        
        return nombres_archivos

    except Exception as e:
        print(f"Ocurrió un error al acceder al bucket: {e}")
        return []


In [ ]:
# Esta función funciona moderadamente bien, pero no es perfecta.
# Por lo mientras, esta va a ser mi función 'segura', es decir, que si algo falla
# en las pruebas, esta es la que voy a usar para hacer los reportes de Gemini.

def reporteGemini(uri, marca_a_buscar, temperatura=0.0):

    """
    Analiza el video y te devuelve la data de las apariciones de la marca
    """

    # --- Configuración del modelo ---

    # Construimos la uri completa (solo tenemos el nombre del archivo)
    uri = 'gs://vboxiooof/Videos/Videos_Segmentados/'+uri


    # Preparamos al video en el formato que puede consumir Gemini
    video = Part.from_uri(uri=uri, mime_type='video/mkv')

    # Configuramos otros parámetros del modelo de Gemini
    generation_config = {'temperature': temperatura} 

    # Definimos el system prompt
    # Reemplaza tu system_instruction con esta nueva versión más precisa

    # Reemplaza tu system_instruction con esta versión HÍBRIDA


    # NOTA: Poner el en prompt que investigue la marca

    system_instruction = """
Eres un analista de medios y publicidad de élite. Tu análisis debe ser contextual y preciso, siguiendo un proceso de decisión en dos etapas.

---
**ETAPA 1: ANÁLISIS DEL CONTEXTO GENERAL DEL CLIP**
---
Primero, evalúa la naturaleza general del video completo.
- ¿Es una **Transmisión en Vivo** del partido, mostrando el juego, a los jugadores en la cancha o a los fans en las gradas, sobre la cual se superponen gráficos?
- ¿O es un **Anuncio Producido**, que interrumpe el juego para contar una historia o mensaje publicitario con edición cinematográfica, música y actores, aunque muestre escenas de fútbol?

**Esta primera evaluación es la más importante.**

---
**ETAPA 2: CLASIFICACIÓN BASADA EN EL CONTEXTO**
---
Una vez determinado el contexto, aplica la siguiente regla:

1.  **Si el contexto es un 'Anuncio Producido':**
    La respuesta DEBE contener una única entrada para la marca. Usa la categoría **'Anuncio'** para la duración completa del clip. **NO reportes las apariciones individuales** de la marca dentro del propio comercial (en vallas, playeras, etc.).

2.  **Si el contexto es una 'Transmisión en Vivo':**
    Entonces, y solo entonces, procede a identificar y reportar cada aparición individual de la marca, eligiendo la categoría más apropiada de la siguiente lista:
    - **'Virtual'**: Gráfico generado por computadora superpuesto al campo mientras el juego continúa viéndose de fondo (ej. la botella de Yakult flotando).
    - **'Valla'**: La marca en las vallas LED físicas que rodean el campo.
    - **'Spot'**: Un gráfico que se sobrepone a la imagen sin estar integrado en el campo.
    - **'Banner'**: Un gráfico en los bordes de la pantalla.
    - **'Marcador'**: La marca junto al marcador de resultado.
    - **'Playera'**: El logo en la camiseta.
    - **'Menciones verbales'**: En comentarios o anuncios



### Criterio de Identificación Estricto
IMPORTANTE: Solo reporta una aparición de la marca si el logo o texto es COMPLETA y PERFECTAMENTE identificable. 
No reportes elementos que sean borrosos, estén parcialmente ocultos, o donde el logo o texto no se pueda distinguir con claridad absoluta. 
Abstente de hacer inferencias basadas en colores o formas similares si el identificador de la marca no es nítido y discernible.

    

### Formato de Salida Obligatorio
- Tu respuesta debe ser únicamente un objeto JSON con la clave "apariciones".
- Si no hay apariciones, devuelve: `{"apariciones": []}`.
- No incluyas explicaciones. Solo el JSON.

**Ejemplo de la estructura exacta que debes seguir:**
```json
{
  "apariciones": [
    {
      "marca": "Coca",
      "categoria": "Valla",
      "inicio": "00:11",
      "fin": "00:15"
    }
  ]
}

### Notas importantes
Los clips duran en total 16 segundos, entonces no puedes decir algo del estilo:
```json
{
  "apariciones": [
    {
      "marca": "Coca",
      "categoria": "Valla",
      "inicio": "00:19",
      "fin": "00:22"
    }
  ]
}

Si detectas más de una aparición en un solo clip, ten mucho cuidado con los tiempos.

- Otra cosa importante, es que si el logo de la marca no se ve con claridad, no lo reportes. Es decír, si no puedes distinguirlo perfectamente bien, no infieras que lo estás viendo
  tan solo porque haya algo con colores parecidos (o algo por el estilo).
"""

    # Definimos la marca
    # marca_a_buscar = 'Caliente.mx'

    
    # Definimos el prompt
    prompt = "Busca en el video adjunto todas las apariciones (si es que existen) de la siguiente marca: "

    model = GenerativeModel('gemini-2.5-pro', system_instruction=system_instruction)


    # --- Hacemos la petición ---

    contenido = [video, prompt, marca_a_buscar]

    # Le pasamos a Gemini el contenido
    respuesta = model.generate_content(

        contents=contenido,
        generation_config=generation_config
        
    )


    # --- Retornamos la respuesta ---
    return respuesta.text

In [52]:
##### BORRAR #####
# Primera prueba modificando la función reporteGemini

def reporteGemini(uri, temperatura=0.0):

    """
    Analiza el video y te devuelve la data de las apariciones de la marca
    """

    # --- Configuración del modelo ---

    # Construimos la uri completa (solo tenemos el nombre del archivo)
    uri = 'gs://vboxiooof/Videos/Videos_Segmentados/'+uri


    # Preparamos al video en el formato que puede consumir Gemini
    video = Part.from_uri(uri=uri, mime_type='video/mkv')

    # Configuramos otros parámetros del modelo de Gemini
    # generation_config = {'temperature': temperatura} 
    generation_config = {
      'temperature': temperatura,
      'top_p': 1,
      'top_k': 1
    }

    # Definimos el system prompt
    # Reemplaza tu system_instruction con esta nueva versión más precisa

    # Reemplaza tu system_instruction con esta versión HÍBRIDA

    system_instruction = """
Eres un sistema de IA experto y altamente preciso, especializado en la detección de apariciones de la marca **'Caliente.mx'** en videoclips de 16 segundos. Tu única tarea es devolver un objeto JSON con las apariciones detectadas.

---
### **Regla de Oro: Cero Inferencias y Máxima Precisión**
---
Tu objetivo principal es **EVITAR FALSOS POSITIVOS**. La precisión es más importante que la detección. Sigue esta regla de forma estricta:

- **SOLO reporta la marca 'Caliente.mx' si su logotipo o el texto "Caliente.mx" es 100% nítido, completo y perfectamente legible.**
- **NO infieras NUNCA.** Si tienes la más mínima duda, no reportes la aparición.
- **PROHIBIDO INFERIR POR:**
    - **Colores similares:** Ignora cualquier elemento rojo si el logo no es explícito.
    - **Temática similar:** Ignora otras marcas de apuestas o casinos. Solo 'Caliente.mx' es relevante.
    - **Logos borrosos o parciales:** Si está cortado, desenfocado o no se distingue con claridad absoluta, NO es una aparición válida.

---
### **ETAPA 1: ANÁLISIS DEL CONTEXTO DEL VIDEO**
---
Primero, evalúa la naturaleza general del video completo.

- **¿Es una Transmisión en Vivo?** Muestra el juego, jugadores, o gradas, y los gráficos de la marca se superponen a esta imagen en tiempo real.
- **¿Es un Anuncio Producido?** Es un comercial cinematográfico que interrumpe el juego. Tiene su propia narrativa, música y edición, aunque use imágenes de fútbol.

**Esta primera evaluación es crucial y determina todo el proceso siguiente.**

---
### **ETAPA 2: CLASIFICACIÓN Y REPORTE**
---
Una vez determinado el contexto, aplica la siguiente lógica:

**1. Si el contexto es un `Anuncio Producido`:**
La respuesta DEBE contener una única entrada. Usa la categoría **`Anuncio`** para la duración completa del clip. NO reportes las apariciones individuales de la marca dentro del propio comercial.

**2. Si el contexto es una `Transmisión en Vivo`:**
Procede a identificar y reportar cada aparición individual de la marca, eligiendo la categoría más apropiada de la siguiente lista. Sé muy cuidadoso y preciso con los tiempos de inicio y fin.

- **`Anuncio`**: (Exclusivo para `Anuncio Producido`).
- **`Cintillo`**: Una banda o franja gráfica horizontal simple.
- **`Marco en L`**: Un gráfico que ocupa un borde lateral Y el borde inferior de la pantalla.
- **`Segmento Patrocinado`**: Un gráfico complejo que ocupa gran parte de la pantalla, a menudo con datos, estadísticas o la imagen de un comentarista.
- **`Valla`**: La marca aparece en las vallas LED físicas que rodean el campo.
- **`Publicidad Virtual / Tapete`**: Anuncio insertado digitalmente sobre la superficie del campo (césped) para que parezca que está pintado ahí.
- **`Marcador`**: La marca aparece integrada o junto al marcador de resultado del partido.
- **`Playera`**: El logo en la camiseta de un jugador o miembro del cuerpo técnico.
- **`Mención Verbal`**: Un comentarista menciona la marca "Caliente" o "Caliente.mx" de forma audible.

---
### **FORMATO DE SALIDA OBLIGATORIO**
---
- Tu respuesta debe ser **únicamente un objeto JSON** con la clave `apariciones`.
- Si no hay apariciones que cumplan los estrictos criterios de identificación, devuelve un arreglo vacío.
- **No incluyas explicaciones, texto adicional o comentarios.** Solo el JSON.

**Ejemplo de estructura para Transmisión en Vivo:**
```json
{
  "apariciones": [
    {
      "marca": "Caliente.mx",
      "categoria": "Marco en L",
      "inicio": "00:02",
      "fin": "00:07"
    },
    {
      "marca": "Caliente.mx",
      "categoria": "Valla",
      "inicio": "00:11",
      "fin": "00:15"
    }
  ]
}

### Notas importantes
Los clips duran en total 16 segundos, entonces no puedes decir algo del estilo:
```json
{
  "apariciones": [
    {
      "marca": "Caliente.mx",
      "categoria": "Valla",
      "inicio": "00:19",
      "fin": "00:22"
    }
  ]
}

Si detectas más de una aparición en un solo clip, ten mucho cuidado con los tiempos.
"""

    # Definimos la marca
    # marca_a_buscar = 'Caliente.mx'

    
    # Definimos el prompt
    # prompt = "Busca en el video adjunto todas las apariciones (si es que existen) de la siguiente marca: "
    prompt = " "

    model = GenerativeModel('gemini-2.5-pro', system_instruction=system_instruction)


    # --- Hacemos la petición ---

    contenido = [video, prompt]

    # Le pasamos a Gemini el contenido
    respuesta = model.generate_content(

        contents=contenido,
        generation_config=generation_config
        
    )


    # --- Retornamos la respuesta ---
    return respuesta.text

In [58]:
def estandarizar_respuesta(texto_json):
    """
    Limpia y ordena la respuesta JSON para garantizar una salida consistente.
    """
    try:
        # --- PASO 1: Limpiar el texto de la respuesta ---
        # Quita los marcadores de bloque de código y espacios en blanco de los extremos
        texto_limpio = texto_json.strip().strip('```').strip('json').strip()

        # --- PASO 2: Convertir el texto limpio a un objeto Python ---
        data = json.loads(texto_limpio)
        
        # --- PASO 3: Ordenar la lista 'apariciones' por 'inicio' y 'categoria' ---
        if 'apariciones' in data and data['apariciones']:
            data['apariciones'] = sorted(data['apariciones'], key=lambda x: (x.get('inicio', ''), x.get('categoria', '')))
            
        # --- PASO 4: Convertir el objeto de nuevo a un string JSON formateado ---
        return json.dumps(data, indent=2, ensure_ascii=False)
        
    except (json.JSONDecodeError, AttributeError) as e:
        # Si la respuesta no es un JSON válido o tiene algún problema,
        # devuelve el texto original para que puedas depurarlo.
        print(f"Error al procesar JSON: {e}")
        return texto_json

In [14]:
def pasar_json(respuesta_gemini: str):
    """
    Limpia la respuesta de texto de Gemini y la convierte en un objeto JSON.
    
    Args:
        respuesta_gemini: La cadena de texto cruda devuelta por `respuesta.text`.

    Returns:
        Un diccionario o lista de Python, o None si ocurre un error.
    """
    try:
        # 1. Limpia el string de los marcadores de Markdown y espacios en blanco
        # Usamos removeprefix y removesuffix que son más seguros que replace
        json_limpio_str = respuesta_gemini.strip().removeprefix('```json').removesuffix('```').strip()
        
        # 2. Convierte el string limpio en un objeto JSON (diccionario/lista)
        return json.loads(json_limpio_str)
        
    except json.JSONDecodeError:
        print("Error: La respuesta de Gemini no pudo ser decodificada como JSON válido.")
        return None
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        return None

# **Pruebas**

### **Primer tiempo**

##### Pruebas con mi sistem prompt 

In [ ]:
# Marcamos el inicio del programa
start_time = time.time()


# Obtenemos la lista de videos
uris = obtener_nombres_de_archivos(nombre_bucket=BUCKET_NAME, nombre_carpeta=VIDEO_FOLDER_PATH)

uris

# data = [] # Aquí vamos a guardar los JSON


for i in range(78, 98):
    print(f'Vamos en {uris[i]}')
    data.append( ['gs://vboxiooof/Videos/Videos_Segmentados/'+uris[i], pasar_json(reporteGemini(uris[i]))] ) # Guardamos tuplas (uri, info)
    time.sleep(10) # Esperamos 10 segundos entre cada petición para evitar problemas de rate limiting


end_time = time.time()


print(f'El proceso duró: {end_time - start_time}')

Vamos en mexicosta_segment_058.mkv
Vamos en mexicosta_segment_059.mkv
Vamos en mexicosta_segment_060.mkv
Vamos en mexicosta_segment_061.mkv
Vamos en mexicosta_segment_062.mkv
Vamos en mexicosta_segment_063.mkv
Vamos en mexicosta_segment_064.mkv
Vamos en mexicosta_segment_065.mkv
Vamos en mexicosta_segment_066.mkv
Vamos en mexicosta_segment_067.mkv
Vamos en mexicosta_segment_068.mkv
Vamos en mexicosta_segment_069.mkv
Vamos en mexicosta_segment_070.mkv
Vamos en mexicosta_segment_071.mkv
Vamos en mexicosta_segment_072.mkv
Vamos en mexicosta_segment_073.mkv
Vamos en mexicosta_segment_074.mkv
Vamos en mexicosta_segment_075.mkv
Vamos en mexicosta_segment_076.mkv
Vamos en mexicosta_segment_077.mkv
El proceso duró: 501.1669900417328


In [103]:
data

[['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_000.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_001.mkv',
  {'apariciones': [{'marca': 'Caliente.mx',
     'categoria': 'Valla',
     'inicio': '00:02',
     'fin': '00:06'}]}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_002.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_003.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_004.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_005.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_006.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_007.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_008.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmenta

In [100]:
print(estandarizar_respuesta(reporteGemini('mexicosta_segment_057.mkv')))

{
  "apariciones": [
    {
      "marca": "Caliente.mx",
      "categoria": "Marcador",
      "inicio": "00:00",
      "fin": "00:06"
    }
  ]
}


In [83]:
print(estandarizar_respuesta(reporteGemini('mexicosta_segment_002.mkv')))

{
  "apariciones": []
}


In [28]:
data

[['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_000.mkv',
  {'apariciones': [{'marca': 'Caliente',
     'categoria': 'Valla',
     'inicio': '00:11',
     'fin': '00:15'}]}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_001.mkv',
  {'apariciones': [{'marca': 'Caliente',
     'categoria': 'Valla',
     'inicio': '00:01',
     'fin': '00:06'}]}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_002.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_003.mkv',
  {'apariciones': [{'marca': 'Caliente',
     'categoria': 'Anuncio',
     'inicio': '00:00',
     'fin': '00:15'}]}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_004.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_005.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_006.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mex